In [11]:
import os

import dlib
import cv2
import numpy as np
import math
from sklearn.svm import SVC
import pickle
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import glob
import random

In [12]:
emotions = ["anger", "contempt","disgust", "fear", "happy", "sadness", "surprise"]
detector = dlib.get_frontal_face_detector()
model = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

In [13]:
# Set the classifier as a support vector machines with linear kernel
# clf = SVC(C=0.01, kernel='linear', decision_function_shape='ovo', probability=True)
clf = SVC(kernel='linear', probability=True, tol=1e-5, verbose = False)

In [14]:
def get_landmarks(image):
    detections = detector(image, 1)
    # For all detected face instances individually
    for k, d in enumerate(detections):
        shape = model(image, d)# Get facial landmarks with prediction model
        xpoint = []
        ypoint = []
        for i in range(0, 68):
            xpoint.append(float(shape.part(i).x))
            ypoint.append(float(shape.part(i).y))

        # Center points of both axis
        xcenter = np.mean(xpoint)
        ycenter = np.mean(ypoint)
        # Calculate distance between particular points and center point
        xdistcent = [(x-xcenter) for x in xpoint]
        ydistcent = [(y-ycenter) for y in ypoint]

        # Prevent divided by 0 value
        if xpoint[11] == xpoint[14]:
            angle_nose = 0
        else:
            # Point 14 is the tip of the nose, point 11 is the top of the nose brigde
            angle_nose = int(math.atan((ypoint[11]-ypoint[14])/(xpoint[11]-xpoint[14]))*180/math.pi)

        # Get offset by finding how the nose brigde should be rotated to become perpendicular to the horizontal plane
        if angle_nose < 0:
            angle_nose += 90
        else:
            angle_nose -= 90

        landmarks = []
        for cx, cy, x, y in zip(xdistcent, ydistcent, xpoint, ypoint):
            # Add the coordinates relative to the centre of gravity
            landmarks.append(cx)
            landmarks.append(cy)

            # Get the euclidean distance between each point and the centre point (the vector length)
            meanar = np.asarray((ycenter,xcenter))
            centpar = np.asarray((y,x))
            dist = np.linalg.norm(centpar-meanar)

            # Get the angle the vector describes relative to the image, corrected for the offset that the nosebrigde
            # has when the face is not perfectly horizontal
            if x == xcenter:
                angle_relative = 0
            else:
                angle_relative = (math.atan(float(y-ycenter)/(x-xcenter))*180/math.pi) - angle_nose
            landmarks.append(dist)
            landmarks.append(angle_relative)

    if len(detections) < 1:
        # In case no case selected, print "error" values
        landmarks = "error"
    return landmarks

In [15]:
def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    path = os.path.join('CK+_Complete/', emotion)
    path2 = os.path.join(path,'*')
    files = glob.glob(path2)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

In [16]:
def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            #image_preprocessing(no need to crop the images of the dataset, already cropped !)
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            landmarks_vec = get_landmarks(gray)
            training_data.append(landmarks_vec)
            training_labels.append(emotion)
    
        for item in prediction: #repeat above process for prediction set
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            landmarks_vec = get_landmarks(gray)
            prediction_data.append(landmarks_vec)
            prediction_labels.append(emotion)

    return training_data, training_labels, prediction_data, prediction_labels

In [17]:
def create_model():
    print("Marking set")
    X_train, y_train, X_test, y_test = make_sets()

    # Turn the training set into a numpy array for the classifier
    np_X_train = np.array(X_train)
    np_y_train = np.array(y_train)
    # Train SVM
    print("Training SVM Classifier")
    clf.fit(np_X_train, np_y_train)

    np_X_test = np.array(X_test)
    np_y_test = np.array(y_test)
    # Use score() function to get accuracy
    print("Getting accuracy score --")
    pred_accuracy = clf.score(np_X_test, np_y_test)
    test_pred = clf.predict(np_X_test)

    print("Test Accuracy: ", pred_accuracy)

    print(confusion_matrix(np_y_test, test_pred))
    print(classification_report(np_y_test, test_pred))

    return pred_accuracy

In [18]:
if __name__ == '__main__':
    accuracy = create_model()
    print('Accuracy = ', accuracy * 100, 'percent')
    model_file = os.path.join('models', 'model1.pkl')
    try:
        os.remove(model_file)
    except OSError:
        pass
    output = open(model_file, 'wb')
    pickle.dump(clf, output)
    output.close()

Marking set
Training SVM Classifier
Getting accuracy score --
Test Accuracy:  0.981651376146789
[[45  0  0  0  0  0  0]
 [ 0 16  0  0  0  2  0]
 [ 1  0 58  0  0  0  0]
 [ 0  0  0 25  0  0  0]
 [ 0  0  0  0 69  0  0]
 [ 0  0  0  0  0 28  0]
 [ 1  0  0  1  0  1 80]]
              precision    recall  f1-score   support

       anger       0.96      1.00      0.98        45
    contempt       1.00      0.89      0.94        18
     disgust       1.00      0.98      0.99        59
        fear       0.96      1.00      0.98        25
       happy       1.00      1.00      1.00        69
     sadness       0.90      1.00      0.95        28
    surprise       1.00      0.96      0.98        83

    accuracy                           0.98       327
   macro avg       0.97      0.98      0.97       327
weighted avg       0.98      0.98      0.98       327

Accuracy =  98.1651376146789 percent
